In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark import SQLContext

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [3]:
# Supongo que TIMESTAMP es de la forma 'yyyy-mm-dd'
rddMed = sc.textFile('dataParcial2019.csv').map(lambda x: (x.split(',')[0], int(x.split(',')[1]), int(x.split(',')[2])))
rddMed.take(5)

[('18-07-20', 3, -3),
 ('18-02-20', 1, -1),
 ('18-09-18', 2, 11),
 ('18-07-15', 3, 16),
 ('18-11-21', 2, 13)]

In [4]:
# Me creo un rdd de la forma (id_base, ('año-mes', tempPromedio)) para todas las mediciones en un mes y base
tempPorMes = rddMed.map(lambda x: (str(x[1])+'-'+x[0][:5], (x[2], 1))).\
             reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])).\
             map(lambda x: (int(x[0].split('-')[0]), (x[0].split('-')[1] + '-' + x[0].split('-')[2], x[1][0]/x[1][1])))
tempPorMes.take(5)

[(3, ('18-03', 7.5)),
 (3, ('18-05', 9.5)),
 (2, ('18-10', 7.166666666666667)),
 (1, ('18-07', 5.0)),
 (1, ('18-04', -5.0))]

In [5]:
# Agrupo por id_base para tener una lista ordenada con todas las temperaturas para una misma base
# Antes de agrupar borro todos los elementos que tienen un mes-año que no me interesa
listasTemp = tempPorMes.filter(lambda x: x[1][0].split('-')[0] == '18' or x[1][0] == '17-12').\
groupByKey().map(lambda x: (x[0], list(x[1]))).map(lambda x: (x[0], sorted(x[1])))
listasTemp.take(5)

[(2,
  [('17-12', 9.0),
   ('18-01', 0.4),
   ('18-02', 8.5),
   ('18-03', 2.6),
   ('18-04', 7.5),
   ('18-05', -3.0),
   ('18-06', 4.0),
   ('18-07', 10.0),
   ('18-08', 6.5),
   ('18-09', 14.666666666666666),
   ('18-10', 7.166666666666667),
   ('18-11', 12.666666666666666)]),
 (3,
  [('17-12', 2.3333333333333335),
   ('18-01', 5.25),
   ('18-02', 4.0),
   ('18-03', 7.5),
   ('18-04', 3.0),
   ('18-05', 9.5),
   ('18-07', 9.166666666666666),
   ('18-08', 7.0),
   ('18-09', 8.0),
   ('18-12', 12.0)]),
 (1,
  [('17-12', 2.0),
   ('18-02', 5.75),
   ('18-03', 7.0),
   ('18-04', -5.0),
   ('18-05', 7.5),
   ('18-06', 6.5),
   ('18-07', 5.0),
   ('18-08', 15.5),
   ('18-10', 9.25),
   ('18-11', 3.6),
   ('18-12', 10.0)])]

In [7]:
#Ahora me creo un rdd con (id_base,lista) donde lista es una lista de las variaciones de temperatura para cada mes
listasVar = listasTemp.map(lambda x: (x[0], [(((x[1][i][1]-x[1][i-1][1])*100)/x[1][i-1][1]) for i in range(1, 13)]))
listasVar.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 1 times, most recent failure: Lost task 0.0 in stage 14.0 (TID 9, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/tomyrd/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/tomyrd/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/tomyrd/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/tomyrd/anaconda3/lib/python3.7/site-packages/pyspark/rdd.py", line 1354, in takeUpToNumLeft


In [ ]:
# Me quedo solo con los elementos que tengan un numero mayor a 30 en sus listas
def contiene_mayores_a_30(lista):
    for num in lista:
        if num > 30:
            return True
    return False

listasVar = listasVar.filter(lambda x: any(num > 30 for num in x[1]))